# Description

Reconcile portfolio and target positions: the ones reconstructed from fills vs ones that are logged during a prod system run.

# Imports

In [1]:
%load_ext autoreload
%autoreload 2

import logging
import os

import pandas as pd

import core.config as cconfig
import dataflow_amp.system.Cx as dtfamsysc
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hpandas as hpandas
import helpers.hprint as hprint
import im_v2.common.universe as ivcu
import oms.broker.ccxt.ccxt_aggregation_functions as obccagfu
import oms.broker.ccxt.ccxt_execution_quality as obccexqu
import oms.broker.ccxt.ccxt_logger as obcccclo
import oms.order.order_converter as oororcon
import reconciliation.sim_prod_reconciliation as rsiprrec

/app/amp/core/signal_processing/decorrelation.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.9/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-8baa1c38-619e-438e-9a0a-ef2bdf669c99.json'
INFO  # Git
  branch_name='CmTask6627_Load_system_config_as_Config'
  hash='60244aa39'
  # Last commits:
    *   60244aa39 Dan      Merge branch 'master' into CmTask6627_Load_system_config_as_Config (83 minutes ago) Tue Mar 26 12:46:56 2024  (HEAD -> CmTask6627_Load_system_config_as_Config, origin/CmTask6627_Load_system_config_as_Config)
    |\  
    | * 9fbd60edc Sameep Pote CmTask7553_Adapt_the_Broker_portfolio_reconciliation_notebook_for_a_bid_ask_model_1_2 (#7629) (   2 hours ago) Tue Mar 26 12:09:51 2024  (origin/master, origin/HEAD)
    | * ce6073700 Sonaal   fix (#7670)                                                       (  16 hours ago) Mon Mar 25 21:54:23 2024           
# Machine info
  system=Linux
  node name=31459fedca06
  release=5.15.0-1052-aws
  version=#57~20.04.1-Ubuntu SMP Mon Jan 15 17:04:56 UTC 2024
  machine

# Functions

In [3]:
# TODO(Dan): Move to a lib.
def verify_no_trades_for_missing_assets(
    reconstructed_df: pd.DataFrame,
    prod_df: pd.DataFrame,
    type_: str,
    share_asset_ids_with_no_fills: float,
) -> None:
    """
    Check that there were no executed/target trades for asset ids that are
    missing in the reconstructed df.

    :param reconstructed_df: data reconstructed from fills/parent orders
    :param prod_df: data loaded from prod run logs directly
    :param type_: data type, "portfolio" or "target_positions"
    :param share_asset_ids_with_no_fills: threshold for acceptable share
        of missing asset ids
    """
    # Set column names to check the absence of trades.
    if type_ == "portfolio":
        trades_col = "executed_trades_shares"
        shares_col = "executed_trades_notional"
    elif type_ == "target_positions":
        trades_col = "target_trades_shares"
        shares_col = "target_trades_notional"
    else:
        raise ValueError("Unsupported `type_`=%s", type_)
    missing_asset_ids = set(prod_df.columns.levels[1]) - set(
        reconstructed_df.columns.levels[1]
    )
    num_missing_asset_ids = len(missing_asset_ids)
    if num_missing_asset_ids > 0:
        # Check that the share of missing asset ids is not greater
        # than an arbitrary threshold.
        num_asset_ids = len(prod_df.columns.levels[1])
        hdbg.dassert_lte(
            num_missing_asset_ids / num_asset_ids,
            share_asset_ids_with_no_fills,
        )
        for asset_id in missing_asset_ids:
            _LOG.info("Missing asset id = %s", asset_id)
            # Check trades in shares.
            trades_shares_sum = (
                prod_df.xs(asset_id, axis=1, level=1, drop_level=False)[
                    trades_col
                ]
                .abs()
                .sum()
                .loc[asset_id]
            )
            # TODO(Dan): Consider to use `hpandas.dassert_approx_eq()`
            #  to account for a floating point error.
            hdbg.dassert_eq(
                0,
                trades_shares_sum,
                msg="Asset id='%s' with no positions/fills has executed/target trades in shares='%s' in total, while 0 is the expected value"
                % (asset_id, trades_shares_sum),
            )
            # Ideally zero trades in shares implies no notional trades,
            # but checking both in case of any data inconsistency.
            trades_notional_sum = (
                prod_df.xs(asset_id, axis=1, level=1, drop_level=False)[
                    shares_col
                ]
                .abs()
                .sum()
                .loc[asset_id]
            )
            hdbg.dassert_eq(
                0,
                trades_shares_sum,
                msg="Asset id='%s' with no with no positions/fills has executed/target nominal trades ='%s' in total, while 0 is the expected value"
                % (asset_id, trades_notional_sum),
            )


def sanity_check_indices_difference(
    reconstructed_df: pd.DataFrame,
    prod_df: pd.DataFrame,
    n_index_elements_to_ignore: int,
) -> None:
    """
    Check that the symmetric indices difference does exceed the threshold.

    :param reconstructed_df: data reconstructed from fills/parent orders
    :param prod_df: data loaded from prod run logs directly
    :param n_index_elements_to_ignore: threshold for number of differing
        indices
    """
    # Sanity check differing indices from reconstructed df.
    reconstructed_df_extra_idx = reconstructed_df.index.difference(prod_df.index)
    hdbg.dassert_lte(len(reconstructed_df_extra_idx), n_index_elements_to_ignore)
    if len(reconstructed_df_extra_idx) > 0:
        # Verify that the difference is a consecutive index.
        hpandas.dassert_strictly_increasing_index(reconstructed_df_extra_idx)
    # Sanity check differing indices from prod df.
    prod_df_extra_idx = prod_df.index.difference(reconstructed_df.index)
    hdbg.dassert_lte(len(prod_df_extra_idx), n_index_elements_to_ignore)
    if len(prod_df_extra_idx) > 0:
        # Verify that the difference is a consecutive index.
        hpandas.dassert_strictly_increasing_index(prod_df_extra_idx)

# Config

In [4]:
config = cconfig.get_config_from_env()
if config:
    # Get config from env when running the notebook via the `run_notebook.py`
    # script, e.g., in the system reconciliation flow.
    _LOG.info("Using config from env vars")
else:
    id_col = "asset_id"
    system_log_dir = (
        "/shared_data/CmTask6032/system_log_dir.manual/process_forecasts"
    )
    # Load pickled SystemConfig.
    config_file_name = "system_config.output.values_as_strings.pkl"
    system_config_dir = system_log_dir.rstrip("/process_forecasts")
    system_config_path = os.path.join(system_config_dir, config_file_name)
    system_config = cconfig.load_config_from_pickle(system_config_path)
    # TODO(Dan): Deprecate after switch to updated config logs CmTask6627.
    hdbg.dassert_in("dag_runner_config", system_config)
    if isinstance(system_config["dag_runner_config"], tuple):
        bar_duration = rsiprrec.extract_bar_duration_from_pkl_config(
            system_config_dir
        )
        universe_version = rsiprrec.extract_universe_version_from_pkl_config(
            system_config_dir
        )
        price_column_name = rsiprrec.extract_price_column_name_from_pkl_config(
            system_config_dir
        )
    else:
        hdbg.dassert_isinstance(system_config, cconfig.Config)
        universe_version = system_config["market_data_config"][
            "im_client_config"
        ]["universe_version"]
        bar_duration_in_secs = system_config["dag_runner_config"][
            "bar_duration_in_secs"
        ]
        bar_duration_in_mins = int(bar_duration_in_secs / 60)
        bar_duration = f"{bar_duration_in_mins}T"
        price_column_name = system_config["portfolio_config"][
            "mark_to_market_col"
        ]
    vendor = "CCXT"
    mode = "trade"
    config_dict = {
        "id_col": id_col,
        "system_log_dir": system_log_dir,
        "ohlcv_market_data": {
            "vendor": vendor,
            "mode": mode,
            "universe_version": universe_version,
        },
        "price_column_name": price_column_name,
        "bar_duration": bar_duration,
        "share_asset_ids_with_no_fills": 0.3,
        "n_index_elements_to_ignore": 2,
        "target_positions_columns_to_compare": [
            "price",
            "holdings_shares",
            "holdings_notional",
            "target_holdings_shares",
            "target_holdings_notional",
            "target_trades_shares",
            "target_trades_notional",
        ],
        "compare_dfs_kwargs": {
            "row_mode": "inner",
            "column_mode": "inner",
            "diff_mode": "pct_change",
            "assert_diff_threshold": 1e-3,
            "log_level": logging.INFO,
        },
    }
    config = cconfig.Config.from_dict(config_dict)
print(config)

WARNING Found Config v1.0 flow: converting
WARNING Found Config v1.0 flow: converting
WARNING Found Config v1.0 flow: converting
id_col: asset_id
system_log_dir: /shared_data/CmTask6032/system_log_dir.manual/process_forecasts
ohlcv_market_data: 
  vendor: CCXT
  mode: trade
  universe_version: v7.1
price_column_name: close
bar_duration: 5T
share_asset_ids_with_no_fills: 0.3
n_index_elements_to_ignore: 2
target_positions_columns_to_compare: ['price', 'holdings_shares', 'holdings_notional', 'target_holdings_shares', 'target_holdings_notional', 'target_trades_shares', 'target_trades_notional']
compare_dfs_kwargs: 
  row_mode: inner
  column_mode: inner
  diff_mode: pct_change
  assert_diff_threshold: 0.001
  log_level: 20


# Reconstruct portfolio and target positions from fills

## Load fills

In [5]:
system_log_dir = config.get_and_mark_as_used(("system_log_dir",))
system_log_dir

'/shared_data/CmTask6032/system_log_dir.manual/process_forecasts'

In [6]:
ccxt_log_reader = obcccclo.CcxtLogger(system_log_dir)

WARNING Path '/shared_data/CmTask6032/system_log_dir.manual/process_forecasts/broker_config.json' doesn't exist!
WARNING Path '/shared_data/CmTask6032/system_log_dir.manual/process_forecasts/args.json' doesn't exist!
WARNING Path '/shared_data/CmTask6032/system_log_dir.manual/process_forecasts/balances' doesn't exist!
WARNING Path '/shared_data/CmTask6032/system_log_dir.manual/process_forecasts/reduce_only/ccxt_child_order_responses' doesn't exist!
WARNING Path '/shared_data/CmTask6032/system_log_dir.manual/process_forecasts/reduce_only/oms_child_orders' doesn't exist!


In [7]:
fills_df = ccxt_log_reader.load_ccxt_trades(convert_to_dataframe=True)

Loading '/shared_data/CmTask6032/system_log_dir.manual/process_forecasts/child_order_fills/ccxt_trades':   0%|…

In [8]:
fills_df.head(3)

,timestamp,datetime,symbol,asset_id,id,order,side,takerOrMaker,price,amount,cost,transaction_cost,fees_currency,realized_pnl,first_timestamp,last_timestamp,first_datetime,last_datetime,buy_count,sell_count,taker_count,maker_count,buy_volume,sell_volume,taker_volume,maker_volume,buy_notional,sell_notional,taker_notional,maker_notional
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-11-08 13:50:32.696000+00:00,2023-11-08 13:50:32.696000+00:00,2023-11-08 13:50:32.696000+00:00,AVAX/USDT:USDT,8717633868,547862323,16259717317,sell,maker,12.827,13.0,166.751,0.033350,USDT,0.0,2023-11-08 13:50:32.696000+00:00,2023-11-08 13:50:32.696000+00:00,2023-11-08 13:50:32.696000+00:00,2023-11-08 13:50:32.696000+00:00,0,1,0,1,0.0,13.0,0.0,13.0,0.0,166.751,0.0,166.751
2023-11-08 13:50:44.609000+00:00,2023-11-08 13:50:44.609000+00:00,2023-11-08 13:50:44.609000+00:00,BAKE/USDT:USDT,1528092593,165767338,4343988167,sell,maker,0.162,394.0,63.828,0.012766,USDT,0.0,2023-11-08 13:50:44.609000+00:00,2023-11-08 13:50:44.609000+00:00,2023-11-08 13:50:44.609000+00:00,2023-11-08 13:50:44.609000+00:00,0,1,0,1,0.0,394.0,0.0,394.0,0.0,63.828,0.0,63.828
2023-11-08 13:50:44.609000+00:00,2023-11-08 13:50:44.609000+00:00,2023-11-08 13:50:44.609000+00:00,BAKE/USDT:USDT,1528092593,165767339,4343988167,sell,maker,0.162,76.0,12.312,0.002462,USDT,0.0,2023-11-08 13:50:44.609000+00:00,2023-11-08 13:50:44.609000+00:00,2023-11-08 13:50:44.609000+00:00,2023-11-08 13:50:44.609000+00:00,0,1,0,1,0.0,76.0,0.0,76.0,0.0,12.312,0.0,12.312


In [9]:
id_col = config.get_and_mark_as_used(("id_col",))
bar_duration = config.get_and_mark_as_used(("bar_duration",))
# Aggregate fills by bar.
bar_fills = obccagfu.aggregate_fills_by_bar(
    fills_df, bar_duration, groupby_id_col=id_col
)

In [10]:
bar_fills.head(3)

first_timestamp                   last_timestamp                   first_datetime                    last_datetime         symbol    asset_id  buy_count  sell_count  taker_count  maker_count  buy_volume  sell_volume  taker_volume  maker_volume  buy_notional  sell_notional  taker_notional  maker_notional       price   amount       cost  transaction_cost  realized_pnl
bar_end_datetime          asset_id                                                                                                                                                                                                                                                                                                                                                                                                    
2023-11-08 13:55:00+00:00 1030828978 2023-11-08 13:52:02.381000+00:00 2023-11-08 13:52:02.381000+00:00 2023-11-08 13:52:02.381000+00:00 2023-11-08 13:52:02.381000+00:00  GMT/USDT:USDT  1030828978          1           0            0            1       428.0        0.000           0.0       428.000       95.5296        0.00000             0.0        95.52960      0.2232  428.000   95.52960          0.019106           0.0
                          1464553467 2023-11-08 13:50:36.100000+00:00 2023-11-08 13:50:36.100000+00:00 2023-11-08 13:50:36.100000+00:00 2023-11-08 13:50:36.100000+00:00  ETH/USDT:USDT  1464553467          0           1            0            1         0.0        0.136           0.0         0.136        0.0000      256.37632             0.0       256.37632   1885.1200    0.136  256.37632          0.051275           0.0
                          1467591036 2023-11-08 13:50:35.872000+00:00 2023-11-08 13:50:35.872000+00:00 2023-11-08 13:50:35.872000+00:00 2023-11-08 13:50:35.872000+00:00  BTC/USDT:USDT  1467591036          0           1            0            1         0.0        0.011           0.0         0.011        0.0000      389.11510             0.0       389.11510  35374.1000    0.011  389.11510          0.077823           0.0

## Load oms parent orders

These are in the internal `amp` format (not the `ccxt` format)

In [11]:
parent_order_df = ccxt_log_reader.load_oms_parent_order(convert_to_dataframe=True)

Loading '/shared_data/CmTask6032/system_log_dir.manual/process_forecasts/oms_parent_orders':   0%|          | …

In [12]:
parent_order_df.head(3)

,creation_timestamp,asset_id,type_,start_timestamp,end_timestamp,curr_num_shares,diff_num_shares,tz,extra_params
order_id,,,,,,,,,
0,2023-11-08 13:50:28.276454+00:00,8717633868,price@custom_twap,2023-11-08 13:50:28.276454+00:00,2023-11-08 13:55:00+00:00,0.0,-13.00,America/New_York,"{'ccxt_symbol': 'AVAX/USDT:USDT', 'stats': {'s..."
1,2023-11-08 13:50:28.276454+00:00,1528092593,price@custom_twap,2023-11-08 13:50:28.276454+00:00,2023-11-08 13:55:00+00:00,0.0,-470.00,America/New_York,"{'ccxt_symbol': 'BAKE/USDT:USDT', 'stats': {'s..."
2,2023-11-08 13:50:28.276454+00:00,8968126878,price@custom_twap,2023-11-08 13:50:28.276454+00:00,2023-11-08 13:55:00+00:00,0.0,-1.18,America/New_York,"{'ccxt_symbol': 'BNB/USDT:USDT', 'stats': {'su..."


## Load OHLCV data

In [13]:
# TODO(Paul): Refine the cuts around the first and last bars.
start_timestamp = bar_fills["first_datetime"].min() - pd.Timedelta(bar_duration)
_LOG.info("start_timestamp=%s", start_timestamp)
end_timestamp = bar_fills["last_datetime"].max() + pd.Timedelta(bar_duration)
_LOG.info("end_timestamp=%s", end_timestamp)

INFO  start_timestamp=2023-11-08 13:45:32.696000+00:00
INFO  end_timestamp=2023-11-08 14:24:09.609000+00:00


In [14]:
universe_version = config.get_and_mark_as_used(
    (
        "ohlcv_market_data",
        "universe_version",
    )
)
vendor = config.get_and_mark_as_used(
    (
        "ohlcv_market_data",
        "vendor",
    )
)
mode = config.get_and_mark_as_used(
    (
        "ohlcv_market_data",
        "mode",
    )
)
# Get asset ids.
asset_ids = ivcu.get_vendor_universe_as_asset_ids(universe_version, vendor, mode)
# Get prod `MarketData`.
db_stage = "preprod"
market_data = dtfamsysc.get_Cx_RealTimeMarketData_prod_instance1(
    asset_ids, db_stage
)
# Load and resample OHLCV data.
ohlcv_bars = dtfamsysc.load_and_resample_ohlcv_data(
    market_data,
    start_timestamp,
    end_timestamp,
    bar_duration,
)
ohlcv_bars.head()

INFO  Unable to fetch DB credentials from environment variables: 
	'POSTGRES_HOST'
	Attempting env file method.
INFO  Unable to fetch DB credentials from env file: 
	
################################################################################
* Failed assertion *
File '/app/amp/im_v2/devops/env/preprod.im_db_config.env' doesn't exist
################################################################################

	Attempting AWS SecretsManager method.
INFO  Fetching secret: preprod.im_data_db
INFO  Created preprod DB connection: 
 None


/app/amp/helpers/hsql_implementation.py:582: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


INFO  fit_intervals=[(Timestamp('2023-11-08 13:45:32.696000+0000', tz='UTC'), Timestamp('2023-11-08 14:24:09.609000+0000', tz='UTC'))]


close                                                                                                                                                                                                                                                                               high                                                                                                                                                                                                                                                                                low                                                                                                                                                                                                                                                                               open                                                                                                                                      \
                          1030828978 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 5115052901 5118394986 6051632686 8717633868 8968126878 9872743573 1030828978 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 5115052901 5118394986 6051632686 8717633868 8968126878 9872743573 1030828978 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 5115052901 5118394986 6051632686 8717633868 8968126878 9872743573 1030828978 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488   
end_timestamp                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
2023-11-08 08:50:00-05:00     0.2226    1884.84    35375.7     0.1618     10.063      4.998     0.2723      3.323     43.203     2.0580     0.6194      0.571     0.1302      5.972     13.778     0.7703    0.07429      2.531     0.3947     0.5102     0.6890      1.447     12.826     246.10      1.450     0.2236    1884.95    35376.5     0.1620     10.094      5.005     0.2724      3.324     43.228     2.0586     0.6268      0.572     0.1302      5.987     13.800     0.7705    0.07435      2.545     0.3948     0.5107     0.6891      1.447     12.831     246.10      1.451     0.2225    1882.32    35340.0     0.1612     10.055      4.995     0.2714      3.309     43.101     2.0536     0.6173      0.570     0.1298      5.971     13.728     0.7689    0.07414      2.523     0.3945     0.5097     0.6881      1.444     12.802     245.73      1.446     0.2236    1883.00    35347.6     0.1612     10.092      4.997     0.2715      3.312     43.117     2.0540     0.6208      0.570     0.1298   
2023-11-08 08:55:00-05:00     0.2238    188

## Compute portfolio

In [15]:
portfolio_from_fills_df = obccexqu.convert_bar_fills_to_portfolio_df(
    bar_fills,
    ohlcv_bars[config["price_column_name"]],
)
hpandas.df_to_str(portfolio_from_fills_df, num_rows=5, log_level=logging.INFO)

holdings_shares                                                                                                                                                                                                                                                                         holdings_notional                                                                                                                                                                                                                                                                         executed_trades_shares                                                                                                                                                                                                                                                                         executed_trades_notional                                                                                                                                                                                                                                                                                pnl                                                                                                                                                                                                                                                                        
asset_id                       1030828978 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 5115052901 5118394986 6051632686 8717633868 8968126878 9872743573        1030828978 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 5115052901 5118394986 6051632686 8717633868 8968126878 9872743573             1030828978 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 5115052901 5118394986 6051632686 8717633868 8968126878 9872743573               1030828978 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 5115052901 5118394986 6051632686 8717633868 8968126878 9872743573 1030828978 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 5115052901 5118394986 6051632686 8717633868 8968126878 9872743573
2023-11-08 13:50:00+00:00             NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN               NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN                    NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN                      NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN       

## Compute target positions

In [16]:
target_positions_from_parent_orders_df = (
    oororcon.convert_order_df_to_target_position_df(
        parent_order_df,
        ohlcv_bars[config["price_column_name"]],
    )
)
hpandas.df_to_str(
    target_positions_from_parent_orders_df, num_rows=5, log_level=logging.INFO
)

holdings_notional                                                                                                                                                                                                                                                                         holdings_shares                                                                                                                                                                                                                                                                              price                                                                                                                                                                                                                                                                         target_holdings_notional                                                                                                                                                                                                                                                                         target_holdings_shares                                                                                                                                                                                                                                                                         target_trades_notional                                                                                                                                                                                                                                                                         target_trades_shares                                                                                                                                                                                                                                                                        
                                 1030828978 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 5115052901 5118394986 6051632686 8717633868 8968126878 9872743573      1030828978 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 5115052901 5118394986 6051632686 8717633868 8968126878 9872743573 1030828978 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 5115052901 5118394986 6051632686 8717633868 8968126878 9872743573               1030828978 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 5115052901 5118394986 6051632686 8717633868 8968126878 9872743573             1030828978 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 5115052901 5118394986 6051632686 8717633868 8968126878 9872743573             1030828978 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 5115052901 5118394986 6051632686 8717633868 8968126878 9872743573           1030828978 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 5115052901 5118394986 6051632686 8717633868 8968126878 9872743573
2023

# Load prod portfolio and target positions

## Portfolio

In [17]:
portfolio_dir = os.path.join(system_log_dir, "portfolio")
#
portfolio_df, _ = rsiprrec.load_portfolio_artifacts(portfolio_dir, bar_duration)
portfolio_df.head()

Loading `holdings_shares` files...:   0%|          | 0/6 [00:00<?, ?it/s]

Loading `holdings_notional` files...:   0%|          | 0/6 [00:00<?, ?it/s]

Loading `executed_trades_shares` files...:   0%|          | 0/6 [00:00<?, ?it/s]

Loading `executed_trades_notional` files...:   0%|          | 0/6 [00:00<?, ?it/s]

Loading `statistics` files...:   0%|          | 0/6 [00:00<?, ?it/s]

holdings_shares                                                                                                                                                                                                                                                                         holdings_notional                                                                                                                                                                                                                                                                            executed_trades_shares                                                                                                                                                                                                                                                                         executed_trades_notional                                                                                          \
                               6051632686 8717633868 2540896331 1528092593 8968126878 1467591036 2476706208 5115052901 3065029174 1891737434 3401245610 1464553467 1966583502 1030828978 2601760471 2683705052 9872743573 2484635488 2099673105 4516629366 2237530510 2425308589 1776791608 2384892553 5118394986        1030828978  1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052  3065029174 3401245610 4516629366 5115052901  5118394986 6051632686 8717633868 8968126878 9872743573             6051632686 8717633868 2540896331 1528092593 8968126878 1467591036 2476706208 5115052901 3065029174 1891737434 3401245610 1464553467 1966583502 1030828978 2601760471 2683705052 9872743573 2484635488 2099673105 4516629366 2237530510 2425308589 1776791608 2384892553 5118394986               6051632686 8717633868 2540896331 1528092593 8968126878 1467591036 2476706208 5115052901 3065029174   
2023-11-08 08:50:00-05:00             0.0        0.0        0.0        0.0       0.00      0.000        0.0        0.0        0.0        0.0        0.0      0.000        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0           0.00000    0.000000    0.00000    0.00000    0.00000    0.00000    0.00000     0.0000     0.0000    0.00000    0.00000    0.00000    0.00000     0.0000        0.0    0.00000    0.000000    0.00000    0.00000      0.000    0.000000     0.0000     0.0000    0.00000     0.0000                    0.0        0.0        0.0        0.0       0.00      0.000        0.0        0.0        0.0        0.0        0.0      0.000        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0                      NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN   
2023-11-08 08:55:00-05:00             0.0      -13.0        0.0     -470.0      -1.18     -0.011        0.0        0.0        0.0        0.0      -13.1     -0.136        0.0      428.0        0.0     -252.0        0.0        0.0       25.0        0.0        0.0        0.0        9.6       79.5        0.0          95.59808 -256.562912 -389.34918  -76.13060   96.69888    0.00000    0.00000    83.1750     0.0000  163.80657    0.00000    0.00000    0.00000     0.0000        0.0 -194.12568    0.000000  -33.05654    0.00000      0.000    0.000000     0.0000  -166.8290 -290.62220     0.0000                    0.0      -13.0        0.0     -470.0      -1.18     -0.011        0.0        0.0        0.0        0.0      -13.1     -0.136        0.0      428.0        0.0     -252.0        0.0        0.0       25.0        0.0        0.0        0.0        9.6       79.5        0.0                    0.000   -166.751      0.000   -76.1400  -290.4452  -389.1151     0.0000     0.0000    0.00000   
2023-11-08 09:00:00-05:00            67.0       15

## Target positions

In [18]:
target_positions_df = rsiprrec.load_target_positions(system_log_dir, bar_duration)
# Select only the columns that are necessary for reconciliation.
target_positions_df = target_positions_df[
    config["target_positions_columns_to_compare"]
]
target_positions_df.columns = target_positions_df.columns.remove_unused_levels()
#
target_positions_df.head()

Loading `target_positions` files...:   0%|          | 0/6 [00:00<?, ?it/s]

price                                                                                                                                                                                                                                                                         holdings_shares                                                                                                                                                                                                                                                                         holdings_notional                                                                                                                                                                                                                                                                            target_holdings_shares                                                                                                     \
asset_id                  1030828978 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 5115052901 5118394986 6051632686 8717633868 8968126878 9872743573      1030828978 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 5115052901 5118394986 6051632686 8717633868 8968126878 9872743573        1030828978  1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052  3065029174 3401245610 4516629366 5115052901  5118394986 6051632686 8717633868 8968126878 9872743573             1030828978 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553   
wall_clock_timestamp                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
2023-11-08 08:50:00-05:00    0.22280   1883.626   35364.40    0.16178    10.0692     4.9996    0.27204     3.3170    43.1862    2.05708    0.62204     0.5710    0.13002     5.9792    13.7772    0.77020   0.074252     2.5338    0.39466    0.51022    0.68876     1.4462    12.8212    245.940     1.4488             0.0      0.000      0.000        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0       0.00        0.0           0.00000    0.000000    0.00000    0.00000    0.00000    0.00000    0.00000     0.0000     0.0000    0.00000    0.00000    0.00000    0.00000     0.0000        0.0    0.00000    0.000000    0.00000    0.00000      0.000    0.000000     0.0000     0.0000    0.00000     0.0000                  428.0     -0.136     -0.011     -470.0        9.6        0.0        0.0       25.0        0.0       79.5   
2023-11-08 08:55:00-05:00    0.22336   1886.492   35395.38    0.161

# Compare portfolio and target positions

## Portfolio

### Columns

In [19]:
portfolio_from_fills_df.columns.levels[0]

Index(['holdings_shares', 'holdings_notional', 'executed_trades_shares', 'executed_trades_notional', 'pnl'], dtype='object')

In [20]:
portfolio_df.columns.levels[0]

Index(['holdings_shares', 'holdings_notional', 'executed_trades_shares', 'executed_trades_notional', 'pnl'], dtype='object')

In [21]:
hdbg.dassert_set_eq(
    portfolio_from_fills_df.columns.levels[0],
    portfolio_df.columns.levels[0],
)

In [22]:
portfolio_from_fills_df.columns.levels[1]

Index([1030828978, 1464553467, 1467591036, 1528092593, 1776791608, 1891737434, 1966583502, 2099673105, 2237530510, 2384892553, 2425308589, 2476706208, 2484635488, 2540896331, 2601760471, 2683705052, 3065029174, 3401245610, 4516629366, 5115052901, 5118394986, 6051632686, 8717633868, 8968126878, 9872743573], dtype='int64', name='asset_id')

In [23]:
portfolio_df.columns.levels[1]

Index([1030828978, 1464553467, 1467591036, 1528092593, 1776791608, 1891737434, 1966583502, 2099673105, 2237530510, 2384892553, 2425308589, 2476706208, 2484635488, 2540896331, 2601760471, 2683705052, 3065029174, 3401245610, 4516629366, 5115052901, 5118394986, 6051632686, 8717633868, 8968126878, 9872743573], dtype='int64')

In [24]:
# Verify that in production portfolio all assets belong to the production universe.
hdbg.dassert_set_eq(
    asset_ids,
    portfolio_df.columns.levels[1],
)
# Having a few `asset_ids` missing in df reconstructed from fills is acceptable
# because they will be missing in case of no trades.
hdbg.dassert_set_eq(
    portfolio_from_fills_df.columns.levels[1],
    portfolio_df.columns.levels[1],
    only_warning=True,
)
# Check that for missing assets ids there were no trades.
type_ = "portfolio"
verify_no_trades_for_missing_assets(
    portfolio_from_fills_df,
    portfolio_df,
    type_,
    config["share_asset_ids_with_no_fills"],
)

### Index

In [25]:
# Set indices to the same timezone.
portfolio_df.index = portfolio_df.index.tz_convert("UTC")
hdbg.dassert_eq(
    portfolio_from_fills_df.index.tz,
    portfolio_df.index.tz,
)

In [26]:
# Report the symmetric difference. There are some extra consecutive index elements
# in the df reconstructed from fills that can be ignored.
hpandas.dassert_indices_equal(
    portfolio_from_fills_df,
    portfolio_df,
    only_warning=True,
)

WARNING * Failed assertion *
cond=False
df1.index.difference(df2.index)=
DatetimeIndex(['2023-11-08 14:20:00+00:00', '2023-11-08 14:25:00+00:00'], dtype='datetime64[ns, UTC]', freq=None)
df2.index.difference(df1.index)=
DatetimeIndex([], dtype='datetime64[ns, UTC]', freq=None)
Continuing as per user request with only_warning=True


In [27]:
sanity_check_indices_difference(
    portfolio_from_fills_df,
    portfolio_df,
    config["n_index_elements_to_ignore"],
)

### Values

In [28]:
# Compare values at intersecting columns / indices.
portfolio_diff_df = hpandas.compare_dfs(
    portfolio_from_fills_df, portfolio_df, **config["compare_dfs_kwargs"]
)
portfolio_diff_df.head(3)

INFO  Max difference factor: inf


AssertionError: 
################################################################################
* Failed assertion *
inf <= 0.001
################################################################################


## Target positions

### Columns

In [ ]:
target_positions_from_parent_orders_df.columns.levels[0]

In [ ]:
target_positions_df.columns.levels[0]

In [ ]:
hdbg.dassert_set_eq(
    config["target_positions_columns_to_compare"],
    target_positions_from_parent_orders_df.columns.levels[0],
)
hdbg.dassert_set_eq(
    config["target_positions_columns_to_compare"],
    target_positions_df.columns.levels[0],
)

In [ ]:
target_positions_df.columns.levels[1]

In [ ]:
target_positions_from_parent_orders_df.columns.levels[1]

In [ ]:
# Verify that in production target positions all assets belong to the production universe.
hdbg.dassert_set_eq(
    asset_ids,
    target_positions_df.columns.levels[1],
)
# Having a few `asset_ids` missing in df reconstructed from parent orders
# is acceptablebecause they will be missing in case of no positions.
hdbg.dassert_set_eq(
    target_positions_from_parent_orders_df.columns.levels[1],
    target_positions_df.columns.levels[1],
    only_warning=True,
)
# Check that for missing assets ids there were no trades.
type_ = "target_positions"
verify_no_trades_for_missing_assets(
    target_positions_from_parent_orders_df,
    target_positions_df,
    type_,
    config["share_asset_ids_with_no_fills"],
)

### Index

In [ ]:
# Set indices to the same timezone.
target_positions_df.index = portfolio_df.index.tz_convert("UTC")
hdbg.dassert_eq(
    target_positions_from_parent_orders_df.index.tz,
    target_positions_df.index.tz,
)

In [ ]:
hpandas.dassert_indices_equal(
    target_positions_from_parent_orders_df,
    target_positions_df,
)

### Values

In [ ]:
# Compare values at intersecting columns / indices.
target_position_diff_df = hpandas.compare_dfs(
    target_positions_from_parent_orders_df,
    target_positions_df,
    **config["compare_dfs_kwargs"],
)
target_position_diff_df.head(3)